In [27]:
import gym

env=gym.make('Taxi-v3',render_mode='ansi').env

env.reset(seed=0)

env=env.unwrapped

arr=env.render()
print(arr)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+




In [28]:
print('Action Space',env.action_space)
print('State Space',env.observation_space)

Action Space Discrete(6)
State Space Discrete(500)


In [29]:
state=env.encode(3,1,2,0)
print('State :',state)
env.s=state
arr=env.render()
print(arr)

State : 328
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+




In [30]:
env.P[env.s]

{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

In [31]:
state=env.encode(3,1,2,0)
env.s=state

epochs=0
penalties=0

frames=[]

done=False

while not done:
    action=env.action_space.sample()

    state,reward,done,_,_=env.step(action)

    if reward == -10:
        penalties+=1

    frames.append({'state':state,'action':action,'reward':reward})
    epochs+=1

print('Timesteps Taken:',epochs)
print('Penalties incurred:',penalties)

Timesteps Taken: 621
Penalties incurred: 198


In [40]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    actions=['South','North','East','West','Pick-up','Drop-of']
    for i ,frame in enumerate(frames):

        env.s=frame['state']

        clear_output(wait=True)

        arr=env.render()
        print(arr)

        print(f'Timestep: {i+1}')
        print(f'State: {frame["state"]}')
        print(f'Action: {frame["action"]}')
        print(f'Reward: {frame["reward"]}')
        sleep(0.1)




In [33]:
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 621
State: 0
Action: 5
Reward: 20


In [34]:
episodes=100
total_epochs=0
total_penalties=0

for _ in range(episodes):
    state=env.reset()

    penalties = 0
    reward = 0
    epochs = 0

    done = False

    while not done:
        action=env.action_space.sample()

        state,reward,done,_,_=env.step(action)

        if reward == -10:
            penalties+=1
        epochs+=1

    
    total_epochs+=epochs
    total_penalties+=penalties

print(f'Result after {episodes} episodes')
print(f'Average timesteps per episode {total_epochs/episodes}')
print(f'Average penalties per episode {total_penalties/episodes}')


Result after 100 episodes
Average timesteps per episode 2902.85
Average penalties per episode 940.06


In [35]:
import numpy as np

states=env.observation_space.n
action=env.action_space.n

print(states,action)

q_table=np.zeros([states,action])
q_table

500 6


array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [36]:
alpha=0.4
gamma=0.6

episodes=100000

for i in range(episodes):
    state=env.reset()[0]

    epochs=0
    penalties=0

    done=False

    while not done:
        action=np.argmax(q_table[state])

        old_value=q_table[state,action]

        next_state, reward, done, _ ,_  = env.step(action)
        next_state=next_state

        next_max=np.max(q_table[next_state])

        new_value=(1-alpha)*old_value+alpha*(reward+gamma*next_max)

        q_table[state,action]=new_value

        state=next_state

print('Finished')




Finished


In [37]:
q_table

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [-2.36974852, -2.37097084, -2.3605998 , -2.37097084, -2.27325184,
        -4.        ],
       [-1.87324883, -1.88031027, -1.90414301, -1.88031027, -0.7504    ,
        -4.        ],
       ...,
       [-1.62175492, -1.22197334, -1.62175492, -1.53640554, -4.        ,
        -4.        ],
       [-2.13270742, -2.14259971, -2.13270742, -2.1410795 , -4.        ,
        -4.        ],
       [-0.736     , -0.736     , -0.736     ,  1.28      , -4.        ,
        -4.        ]])

In [38]:
episodes=100

total_epochs=0
total_penalties=0

for _ in range(episodes):
    state=env.reset()[0]

    penalties=0
    epochs=0
    reward=0

    done=False

    while not done:
        action=np.argmax(q_table[state])

        state,reward,done,_,_=env.step(action)

        if reward == -10:
            penalties+=1
        epochs+=1

    
    total_epochs+=epochs
    total_penalties+=penalties

print(f'Result after {episodes} episodes')
print(f'Average timesteps per episode {total_epochs/episodes}')
print(f'Average penalties per episode {total_penalties/episodes}')


Result after 100 episodes
Average timesteps per episode 13.11
Average penalties per episode 0.0


In [41]:
state=env.encode(3,0,3,0)
print('State:',state)
env.s=state

arr=env.render()
print(arr)

frames=[]

done=False

while not done:
    action=np.argmax(q_table[state])

    state,reward,done,_,_=env.step(action)

    frames.append({'state':state,'action':action,'reward':reward})

print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 15
State: 0
Action: 5
Reward: 20
